In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, get_scheduler
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
import evaluate

In [2]:
# load dataset
raw_datasets = load_dataset("glue", "mrpc") # sst2 : sentiment classification
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# def tokenize_function(example):
#     return tokenizer(example["sentence"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True) # use map to get Dataset object returned.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # collator

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /data/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [3]:
# only sentence, label, idx columns
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [7]:
raw_datasets['train'][0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0}

In [11]:
# input_ids, token_type_ids, attention_mask columns are added
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [5]:
# remove unnecessary columns
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
# tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [15]:
# a dictionary with keys and lists
tokenized_datasets['train'][0]

{'labels': 0,
 'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
tokenized_datasets.set_format("torch")

In [17]:
# values are converted to tensors
tokenized_datasets['train'][0]

{'labels': tensor(0),
 'input_ids': tensor([  101,  5342,  2047,  3595,  8496,  2013,  1996, 18643,  3197,   102]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

In [6]:
batch_size = 32
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=batch_size, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=batch_size, collate_fn=data_collator
)

In [7]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([32]),
 'input_ids': torch.Size([32, 72]),
 'token_type_ids': torch.Size([32, 72]),
 'attention_mask': torch.Size([32, 72])}

In [8]:
# warning은 이제 익숙!
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [24]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape) # NLL loss.

tensor(0.7700, grad_fn=<NllLossBackward0>) torch.Size([32, 2])


In [9]:
# declare optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

In [10]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler( # learning rate scheduler
    "linear",
    optimizer=optimizer, # insert optimizer because optimizer also keeps track of learning rates
    num_warmup_steps=0,
    num_training_steps=num_training_steps, # number of warmup steps
)
print(num_training_steps)

345


In [11]:
# move model to device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [12]:
GPU_NUM = 0
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

In [13]:
progress_bar = tqdm(range(num_training_steps))

model.train() # enable model graph to keep track of gradients
for epoch in range(num_epochs):
    print('Epoch ', epoch)
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()} # move training data to gpu
        
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward() # backpropagate

        optimizer.step() # gradient descent
        lr_scheduler.step() # update lr schedule
        optimizer.zero_grad() # reset gradient
        progress_bar.update(1)

  0%|          | 0/345 [00:00<?, ?it/s]

Epoch  0
Epoch  1
Epoch  2


In [15]:
metric = evaluate.load("glue", "mrpc") # 
model.eval() # eval mode: batchnorm or dropout layers are turned off.
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad(): # deactivate autograd engine.
        outputs = model(**batch) # forward pass for the batch

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1) # make prediction
    metric.add_batch(predictions=predictions, references=batch["labels"]) # accumulate batch

metric.compute()

{'accuracy': 0.8676470588235294, 'f1': 0.9084745762711864}